In [ ]:
!pip install lightgbm

In [1]:
import pandas as pd
import json
import datetime as dt
import numpy as np
## m

## Load Data

In [2]:
def get_one_hot(df, feature):
    one_hot = pd.get_dummies(df[feature])
    one_hot.columns = [feature+'_'+str(i.lower().translate(str.maketrans('áéíóúü','aeiouu'))) for i in one_hot.columns]
    return one_hot

In [3]:
with open('../data/train.json', mode='r') as f:
    data = json.load(f)
list_dict_data = []
for i in data:
    dict_data = {}
    for j in i:
        dict_data[list(j.keys())[0]] = j[list(j.keys())[0]]
    list_dict_data.append(dict_data)
train = pd.DataFrame(list_dict_data)
train.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(train.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

train.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
train['fecha_publicacion_anuncio'] = train['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['fecha_captura_informacion'] = train['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['interacciones'] = train['interacciones'].astype('int32')
train['duracion_anuncio_total'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# train['duracion_anuncio_parcial'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# mask = train['id_anuncio'].astype(str)=='None'
# train.loc[mask, 'id_anuncio'] = train[mask]['id_observacion']
# train['key'] = train['id_anuncio'].astype(str) +'_'+\
#                train['id_conjunto_anuncios'].astype(str) +'_'+\
#                train['id_campania'].astype(str) +'_'+\
#                train['fecha_captura_informacion'].astype(str)
train.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True)
train.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True)
train = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']
target = ['interacciones']

list_df = []
for i in var_categ:
    df = get_one_hot(train, i)
    list_df.append(df)
train_categ = pd.concat(list_df,axis=1)

train_final = train.join(train_categ)
train_final.drop(var_categ, axis=1, inplace=True)
Y = train_final['interacciones']
train_final.drop(['interacciones'], axis=1, inplace=True)
train_final

,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,fuente_direct,fuente_fb + ig,fuente_facebook,fuente_instagram,...,formato_video,nombre_campania_aon,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_000001,2.091253,0.0,0.0,0.0,962.00000,2.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ID_000002,2.080676,0.0,0.0,0.0,926.00000,4.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ID_000003,1.380211,0.0,0.0,0.0,426.00000,5.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ID_000004,4.875708,0.0,0.0,0.0,1246.00000,11.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ID_000005,4.216825,0.0,0.0,0.0,1628.00000,12.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_005558,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ID_005559,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ID_005560,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
# validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
validation.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(validation.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

validation.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
validation['fecha_publicacion_anuncio'] = validation['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['fecha_captura_informacion'] = validation['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['duracion_anuncio_total'] = (validation['fecha_captura_informacion'] - validation['fecha_publicacion_anuncio']).dt.days
# train['duracion_anuncio_parcial'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# mask = train['id_anuncio'].astype(str)=='None'
# train.loc[mask, 'id_anuncio'] = train[mask]['id_observacion']
# train['key'] = train['id_anuncio'].astype(str) +'_'+\
#                train['id_conjunto_anuncios'].astype(str) +'_'+\
#                train['id_campania'].astype(str) +'_'+\
#                train['fecha_captura_informacion'].astype(str)
validation.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True)
validation.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True)
# validation = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']
list_df = []
for i in var_categ:
    df = get_one_hot(validation, i)
    list_df.append(df)
validation_categ = pd.concat(list_df,axis=1)

validation_final = validation.join(validation_categ)
validation_final.drop(var_categ, axis=1, inplace=True)
validation_final = validation_final.set_index('id_observacion')
validation_final['gasto'] = validation_final['gasto'].replace({'40000%':40000})
validation_final['gasto'] = validation_final['gasto'].astype(float)
validation_final

,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,fuente_direct,fuente_fb + ig,fuente_facebook,fuente_instagram,...,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_gastronomia,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_000001,7.351579,0.0,0.0,0.0,2441.00000,13.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_000002,0.814209,0.0,0.0,0.0,346.00000,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_000003,1.356041,0.0,0.0,0.0,604.00000,3.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_000004,6.137025,0.0,0.0,0.0,1973.00000,6.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_000005,7.101953,0.0,0.0,0.0,2334.00000,7.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002374,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_002375,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_002376,461.538461,NaN,NaN,NaN,65934.06593,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
features = list(set(train_final.columns).intersection(set(validation_final.columns)))
train_final = train_final[features]
validation_final = validation_final[features]

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(train_final, Y, test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4449, 41), (1113, 41), (4449,), (1113,))

In [16]:
from lightgbm import LGBMRegressor
learner = LGBMRegressor(n_estimators=1000)
learner.fit(x_train, y_train,  early_stopping_rounds=10, eval_metric="mae",
                eval_set=[(x_train, y_train), (x_test, y_test)], verbose=50)

[50]	training's l1: 135.801	training's l2: 354564	valid_1's l1: 169.146	valid_1's l2: 608042


C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(n_estimators=1000)

In [17]:
validation_final['INTERACCIONES'] = learner.predict(validation_final)
validation_final

C:\Users\usuario\AppData\Local\Temp/ipykernel_4516/1123091501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_final['INTERACCIONES'] = learner.predict(validation_final)


,nombre_campania_gastronomy,objetivo_campania_reach,objetivo_campania_video views,formato_rm display,fuente_fb + ig,formato_key words,nombre_campania_terra lima,fuente_facebook,nombre_campania_aon,visitas_al_75,...,nombre_campania_terra cusco,objetivo_campania_conversions,fuente_youtube,nombre_campania_cusquena,fuente_direct,objetivo_campania_brand awareness,fuente_instagram,visitas_al_50,visitas_al100,INTERACCIONES
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_000001,0,0,0,0,0,0,0,0,1,0.0,...,0,0,0,0,0,0,0,0.0,0.0,69.481876
ID_V_000002,0,0,0,0,0,0,0,0,1,0.0,...,0,0,0,0,0,0,0,0.0,0.0,158.150370
ID_V_000003,0,0,0,0,0,0,0,0,1,0.0,...,0,0,0,0,0,0,0,0.0,0.0,50.194549
ID_V_000004,0,0,0,0,0,0,0,0,1,0.0,...,0,0,0,0,0,0,0,0.0,0.0,33.474002
ID_V_000005,0,0,0,0,0,0,0,0,1,0.0,...,0,0,0,0,0,0,0,0.0,0.0,69.481876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002374,0,0,1,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,NaN,NaN,571.688703
ID_V_002375,0,0,1,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,NaN,NaN,571.688703
ID_V_002376,0,0,1,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,NaN,NaN,571.688703


In [19]:
validation_final = validation_final.reset_index()
validation_final = validation_final[['id_observacion','INTERACCIONES']]
# ID_OBSERVACION, INTERACCIONES.
validation_final.columns = ['ID_OBSERVACION', 'INTERACCIONES']
validation_final

,ID_OBSERVACION,INTERACCIONES
0,ID_V_000001,69.481876
1,ID_V_000002,158.150370
2,ID_V_000003,50.194549
3,ID_V_000004,33.474002
4,ID_V_000005,69.481876
...,...,...
2373,ID_V_002374,571.688703
2374,ID_V_002375,571.688703
2375,ID_V_002376,571.688703
2376,ID_V_002377,571.688703


In [21]:
validation_final.to_csv('../results/test1.csv', index=False)